In [1]:
"""
引入需要的包
"""
# 引入pytorch相关的方法包
import torch
import torch.nn as nn # 构建网络相关函数
import torch.optim as optim # 优化器
from torch.utils.data import DataLoader # 数据加载器
import torch.backends.cudnn as cudnn # cuda配置

# 引入数据集导入、处理包
from torchvision.datasets.mnist import MNIST
import torchvision.transforms as transforms

# 引入集合处理包，用于网络模型中包装网络层的序列
from collections import OrderedDict

# 引入LQA包
import LQA

In [2]:
"""
MNIST 数据集准备
"""
# MNIST 训练集
data_train = MNIST('data/mnist',
                   download=True,
                   transform=transforms.Compose([
                       transforms.Resize((32, 32)),
                       transforms.ToTensor()]))
# MNIST 测试集
data_test = MNIST('data/mnist',
                  train=False,
                  download=True,
                  transform=transforms.Compose([
                      transforms.Resize((32, 32)),
                      transforms.ToTensor()]))
# 将2个数据集（训练集、测试集）加载为读取器
data_train_loader = DataLoader(data_train, batch_size=64, shuffle=True, num_workers=8)
data_test_loader = DataLoader(data_test, batch_size=64, num_workers=8)

In [3]:
"""
构建网络模型
"""
# 示例：一个简单的MLP模型
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        
        in_channels = 1
        self.model = nn.Sequential(OrderedDict([
            ('fc1', nn.Linear(1024, 1000)), # 输入层连接到第1个全连接层，1000个神经元
            ('relu1', nn.ReLU()), # relu激活函数
            ('f2', nn.Linear(1000, 1000)), # 第1个全连接层连接到第2个全连接层，1000个神经元
            ('relu2', nn.ReLU()), # relu激活函数
            ('f3', nn.Linear(1000, 10)) # 连接到输出层
        ]))

    def forward(self, img):
        output = img.view(img.size(0), -1) # 对输入的图像矩阵，先拉伸为一维向量
        output = self.model(output) # 然后传入模型进行训练
        return output

In [4]:
"""
模型训练准备工作：定义一次迭代时，训练、测试的方法
"""

# 设置cuda
device = torch.device("cuda") 
cudnn.benchmark = True 
# 设定pytorch的随机种子
torch.manual_seed(1) 

# 定义模型训练的方法
# 输入一个网络模型、损失函数、优化器、迭代次数，即可进行训练，更新网络参数估计
def train(net, criterion, optimizer, epoch):
    global device
    
    # 进入网络训练模式
    net.train()
    # 遍历训练集的每个 batch
    for i, (images, labels) in enumerate(data_train_loader):
        # 获取每个 batch 的图像（images）及相应标签（labels），并通过【to(device)】放到GPU上
        images, labels = images.to(device), labels.to(device)
        # 优化器梯度归零
        optimizer.zero_grad()
        # 将 images 放入模型，通过模型得到预测的标签
        output = net(images)
        # 将预测的标签和真实标签对比，从而计算loss
        loss = criterion(output, labels)
        # 反向传播，得到梯度
        loss.backward()   
        """
        LQA
        """
        # 代入优化器、模型、损失函数、当前的输入images和labels，以及当前迭代次数，计算出最优学习率
        optimal_lr = LQA.LQA_delta(optimizer, net, criterion, images, labels, epoch)
        # 优化器更新网络参数估计
        optimizer.step()

# 定义模型测试方法
# 输入一个网络模型、损失函数、数据加载器、数据集，
# 即可计算在该数据集上的平均loss、精度accuracy(ACC)
def test(net, criterion, _data_loader, _dataset):
    global device
    # 测试时不需要反向传播更新参数，所以不计算梯度，减小计算开销
    with torch.no_grad():
        # 进入网络评价模式
        net.eval()
        # 为计算精度ACC、loss，准备相应的量
        total_correct = 0
        avg_loss = 0.0
        # 遍历每个 batch 的数据
        for i, (images, labels) in enumerate(_data_loader):
            # 将 batch 数据放到GPU上
            images, labels = images.to(device), labels.to(device)
            # 对网络输入 images 得到预测标签
            output = net(images)
            # 将损失函数的累和加到 avg_loss
            avg_loss += criterion(output, labels).sum()
            # 对每个输出（one-hot向量，例如[0,0,0,...,1]）换算成单个数值（例如 9）
            pred = output.detach().max(1)[1]
            # 通过比对换算后的数值与真实标签是否相等，计算 batch 上预测正确的个数
            total_correct += pred.eq(labels.view_as(pred)).sum()
        # 计算平均 loss
        avg_loss /= len(_dataset)
        # 将平均 loss 的值传递到CPU上
        avg_loss_value = avg_loss.detach().cpu().item()
        # 计算精度ACC （正确预测个数/样本总数）
        acc = float(total_correct) / len(_dataset)
        return [avg_loss_value, acc]    

In [5]:
"""
模型迭代训练
"""
# 生成一个MLP模型，并放到GPU上
net = MLP().to(device)
# 指定损失函数为交叉熵
criterion = nn.CrossEntropyLoss()
# 优化器为SGD，初始学习率为0.1
optimizer = optim.SGD(net.parameters(), lr=1e-1)

    
# 迭代10次
for epoch in range(0, 10):
    # 单次迭代的过程
    # 1.通过训练方法，训练、更新网络
    train(net, criterion, optimizer, epoch) 
    # 2.通过测试方法，记录训练集上的 loss、ACC
    [train_avg_loss, train_acc] = test(net, criterion, data_train_loader, data_train) 
    # 3.通过测试方法，记录测试集上的 loss
    [test_avg_loss, test_acc] = test(net, criterion, data_test_loader, data_test) 

    # 显示当前训练情况
    print('** Epoch [%d] | Train(loss: %f ACC: %.4f) | Test(loss: %f ACC: %.4f)' % 
          (epoch, train_avg_loss, train_acc, test_avg_loss, test_acc))

** Epoch [0] | Train(loss: 0.002823 ACC: 0.9424) | Test(loss: 0.002955 ACC: 0.9382)
** Epoch [1] | Train(loss: 0.001451 ACC: 0.9720) | Test(loss: 0.001664 ACC: 0.9682)
** Epoch [2] | Train(loss: 0.000970 ACC: 0.9816) | Test(loss: 0.001272 ACC: 0.9733)
** Epoch [3] | Train(loss: 0.000704 ACC: 0.9862) | Test(loss: 0.001066 ACC: 0.9786)
** Epoch [4] | Train(loss: 0.000641 ACC: 0.9877) | Test(loss: 0.001071 ACC: 0.9789)
** Epoch [5] | Train(loss: 0.000759 ACC: 0.9846) | Test(loss: 0.001268 ACC: 0.9760)
** Epoch [6] | Train(loss: 0.000537 ACC: 0.9892) | Test(loss: 0.001036 ACC: 0.9776)
** Epoch [7] | Train(loss: 0.000316 ACC: 0.9945) | Test(loss: 0.000893 ACC: 0.9813)
** Epoch [8] | Train(loss: 0.000286 ACC: 0.9947) | Test(loss: 0.000846 ACC: 0.9833)
** Epoch [9] | Train(loss: 0.000325 ACC: 0.9941) | Test(loss: 0.001031 ACC: 0.9791)
